In [2]:
import os
import requests
import json
import concurrent.futures
import pandas as pd
from pydub import AudioSegment

## Генерация переменных в json

In [10]:


df=pd.read_excel('метки без store name.xlsx')
VARIABLES=["client_name", "store_name_1", "store_name_2", "date", "product_name"]
client_name=['Владимир Иванович','Иван Владимирович']
store_name_1=['Мвидео','Эльдорадо']
store_name_2=['Мвидео','Эльдорадо']
date=['двадцать шестого июля', 'первого августа']
product_name=['утюг','телевизор, смартфон и холодильник']

def filter_rows(row):
    return row['name'] not in VARIABLES

def create_hierarchy(row):
    start_var, end_var, name_var=row['start, ms'],row['end, ms'],row['name']
    data=[]


    hierarchy_list=[]
    filtered_df=df[(df['start, ms']>=start_var)&(df['end, ms']<=end_var)&(df['name']!=name_var)]
    for _, other_row in filtered_df.iterrows():
        if(other_row['name'] in VARIABLES):
            if(VARIABLES.index(other_row['name'])==0): data=client_name
            elif(VARIABLES.index(other_row['name'])==1): data=store_name_1
            elif(VARIABLES.index(other_row['name'])==2): data=store_name_2
            elif(VARIABLES.index(other_row['name'])==3): data=date
            else: data=product_name
        hierarchy_list.append((other_row['name'], *data, round((other_row['start, ms']-start_var)*1000), round((other_row['end, ms']-other_row['start, ms'])*1000)))

    output=json.dumps(hierarchy_list, ensure_ascii=False).replace('[', '(').replace(']', ')').replace('\"', "'")
    output=list(output)
    output[0]='['
    output[-1]=']'
    output=''.join(output)
    return output


def get_mark_json(data):
    output=data

    output['hierarchy']=output.apply(create_hierarchy, axis=1)
    output=output[output.apply(filter_rows, axis=1)]

    print(output)

    data_dict={}
    for name in output['name'].unique():
        rows=df[df['name']==name]
        nested_labels=rows['hierarchy'].tolist()
        data_dict[name]={'variables': nested_labels, 'template': rows['template'].tolist()}

    with open('marks.json', 'w', encoding='utf-8') as file:
        json.dump(data_dict, file, ensure_ascii=False)

        
get_mark_json(df)


     start, ms     end, ms                                      name  \
0   104.875465  117.443406    Входящий звонок при доступности анкеты   
2   120.015465  130.585465  Входящий звонок при недоступности анкеты   
3   132.965465  134.085465                                     hello   
5   136.205465  137.175465                              hello_noname   
6   139.135465  147.095465                                   Cantalk   
7   149.175465  159.835465                                  Cantalk1   
8   161.165465  176.607917                                      Q1_1   
11  176.923067  180.986567                                      Q1_2   
12  185.265465  192.565465                                        q1   
13  195.915465  201.115465                                        Q3   
14  203.755465  206.305465                                        Q4   
15  208.035465  212.035465                                  Bye_good   
16  214.785465  219.875465               Tag: Перевести на опера

## изменение частоты дискретизации у аудио (качество плохое)

In [ ]:


def change_sample_rate(input_folder, output_folder, target_sample_rate):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Получить список всех файлов WAV во входной папке
    wav_files = [file for file in os.listdir(input_folder) if file.lower().endswith('.wav')]

    for filename in wav_files:
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename)

        # Загрузить аудиофайл
        sound = AudioSegment.from_file(input_path, format="wav")

        # Изменить частоту дискретизации
        sound = sound.set_frame_rate(target_sample_rate)

        # Сохранить файл с новой частотой дискретизации
        sound.export(output_path, format="wav")


input_folder = "аудио"
output_folder = "аудио для апи"
target_sample_rate = 8000

change_sample_rate(input_folder, output_folder, target_sample_rate)


## автоматизация запросов

In [11]:


def process_request(url, params, input_folder, output_folder, input_file):
    # Загрузить аудиофайл
    input_path=input_folder+'/'+input_file
    
    with open(input_path, "rb") as file:
        
        # files = {"file": (os.path.basename(input_file), file, "audio/wav")}
        binary_data = file.read()

        files = {
            'file': (input_file, open(input_path, 'rb'), 'audio/wav')
        }

        headers = {
            "accept": "application/json"
        }
   
        # Отправить POST-запрос с параметрами и аудиофайлом
        response = requests.post(url, files=files, headers=headers)

        try:
            print(response.json())
        except ValueError:
            # Сохранить ответ в файл
            output_path = os.path.join(output_folder, input_file)
            with open(output_path, "wb") as output_file:
                output_file.write(response.content)
            print("Ответ сохранен в:", output_path)
        else:
            print(f"Произошла ошибка при обработке файла {input_file}.")
            # print(response.json())
            # print(params)
            # print(url)

def main():
    url = "https://paidmethods.mcn.ru/api/private/api/speech/brandVoice"  # Укажите URL для отправки POST-запроса

    with open("marks.json", "r", encoding='utf-8') as params_file:
        parameters = json.load(params_file)

    input_folder = "аудио"
    output_folder = "ответы от апи без store name"
    accountID=121079

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []

        for i in parameters:
            filename=i.replace(':','_')+".wav"
            # print(filename)
            param={}
            param["accountID"]=accountID
            param["variables"]=parameters[i]['variables'][0]
            param["template"]=parameters[i]['template'][0]
            url_params=url+'?accountId='+str(accountID)+'&template='+param["template"]+'&variables='+param["variables"]
            # input_path = os.path.join(input_folder, filename)
            futures.append(executor.submit(process_request, url_params, param, input_folder, output_folder, filename))

        # Дождаться завершения всех запросов
        concurrent.futures.wait(futures)

main()


{'ok': False, 'error': 'unknown'}
Произошла ошибка при обработке файла hello_noname.wav.
{'ok': False, 'error': 'unknown'}
Произошла ошибка при обработке файла Входящий звонок при недоступности анкеты.wav.
Ответ сохранен в: ответы от апи без store name\hello.wav
{'ok': False, 'error': 'unknown'}
Произошла ошибка при обработке файла Tag_ are_you_hear_me.wav.
{'ok': False, 'error': 'unknown'}
Произошла ошибка при обработке файла Q4.wav.
{'ok': False, 'error': 'unknown'}
Произошла ошибка при обработке файла Tag_ Why.wav.
{'ok': False, 'error': 'unknown'}
Произошла ошибка при обработке файла Q1_2.wav.
{'ok': False, 'error': 'unknown'}
Произошла ошибка при обработке файла Tag_ are_you_robot.wav.
Ответ сохранен в: ответы от апи без store name\Входящий звонок при доступности анкеты.wav
{'ok': False, 'error': 'unknown'}
Произошла ошибка при обработке файла Cantalk1.wav.
Ответ сохранен в: ответы от апи без store name\Q1_1.wav
{'ok': False, 'error': 'unknown'}
Произошла ошибка при обработке файл

In [88]:

input_folder = "аудио"
output_folder = "ответы от апи"
accountID=121079
input_file='Входящий звонок при доступности анкеты.wav'
url="https://paidmethods.mcn.ru/api/private/api/speech/brandVoice?accountId=121079&template=Здравствуйте, {client_name}. Это компания {store_name_1}. Мы проверяем качество обслуживания в магазине. Ваш телефон, указан, как контактный. Вы сможете ответить на несколько вопросов? Это займет менее минуты&variables=[('client_name', 'Иван', 'Александр', 613, 1138), ('store_name_1', 'Мвидео', 'Эльдорадо', 2627, 650)]"

input_path=input_folder+'/'+input_file
    
with open(input_path, "rb") as file:
    print("ok")
    # files = {"file": (os.path.basename(input_file), file, "audio/wav")}
    binary_data = file.read()

    files = {
        'file': (input_file, open(input_path, 'rb'), 'audio/wav')
    }

    headers = {
        "accept": "application/json"
    }
   
    # Отправить POST-запрос с параметрами и аудиофайлом
    response = requests.post(url, files=files, headers=headers)

    try:
        print(response.json())
    except ValueError:
        # Сохранить ответ в файл
        output_path = os.path.join(output_folder, input_file)
        with open(output_path, "wb") as output_file:
            output_file.write(response.content)
        print("Ответ сохранен в:", output_path)
    else:
        print(f"Произошла ошибка при обработке файла {input_file}.")
        # print(response.json())
        # print(params)
        # print(url)


ok
Ответ сохранен в: ответы от апи\Входящий звонок при доступности анкеты.wav
